In [1]:
import os
import shutil

original_dataset_dir = '/data/book_practical_pytorch_project/plant_leaf'
classes_list = os.listdir(original_dataset_dir)

In [2]:
classes_list[:3]

['Potato___Early_blight', 'Tomato___Bacterial_spot', 'Tomato___Leaf_Mold']

# 데이터 분할을 위한 폴더 생성

In [8]:
base_dir = './splitted'
os.mkdir(base_dir)

train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
valid_dir = os.path.join(base_dir, 'valid')
os.mkdir(valid_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

for clss in classes_list:
    os.mkdir(os.path.join(train_dir, clss))
    os.mkdir(os.path.join(valid_dir, clss))
    os.mkdir(os.path.join(test_dir, clss))

# 데이터 분할과 클래스별 데이터 수 확인

In [9]:
import math

for clss in classes_list:
    path = os.path.join(original_dataset_dir, clss)
    fnames = os.listdir(path)
    
    train_size = math.floor(len(fnames) * 0.6)
    valid_size = math.floor(len(fnames) * 0.2)
    test_size = math.floor(len(fnames) * 0.2)
    
    train_fnames = fnames[:train_size]
    valid_fnames = fnames[train_size:(train_size+valid_size)]
    test_fnames = fnames[(train_size+valid_size):(train_size+valid_size+test_size)]
    
    def _copy(fnames, dst_dir, dataname):
        for fname in fnames:
            src = os.path.join(path, fname)
            dst = os.path.join(os.path.join(dst_dir, clss), fname)
            shutil.copyfile(src, dst)
        print('{} size for class {}: {}'.format(dataname, clss, len(fnames)))
    _copy(train_fnames, train_dir, 'Train')
    _copy(valid_fnames, valid_dir, 'Valid')
    _copy(test_fnames, test_dir, 'Test')

Train size for class Potato___Early_blight: 600
Valid size for class Potato___Early_blight: 200
Test size for class Potato___Early_blight: 200
Train size for class Tomato___Bacterial_spot: 1276
Valid size for class Tomato___Bacterial_spot: 425
Test size for class Tomato___Bacterial_spot: 425
Train size for class Tomato___Leaf_Mold: 571
Valid size for class Tomato___Leaf_Mold: 190
Test size for class Tomato___Leaf_Mold: 190
Train size for class Tomato___Target_Spot: 842
Valid size for class Tomato___Target_Spot: 280
Test size for class Tomato___Target_Spot: 280
Train size for class Corn___Northern_Leaf_Blight: 591
Valid size for class Corn___Northern_Leaf_Blight: 197
Test size for class Corn___Northern_Leaf_Blight: 197
Train size for class Tomato___healthy: 954
Valid size for class Tomato___healthy: 318
Test size for class Tomato___healthy: 318
Train size for class Tomato___Tomato_Yellow_Leaf_Curl_Virus: 3214
Valid size for class Tomato___Tomato_Yellow_Leaf_Curl_Virus: 1071
Test size fo

# 베이스라인 모델 학습을 위한 준비

In [15]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else "cpu")

BATCH_SIZE = 256
EPOCH = 30

# 변환 및 Dataset 정의
transform_base = transforms.Compose([transforms.Resize((64,64)), transforms.ToTensor()])
train_dataset = ImageFolder(root='./splitted/train', transform=transform_base)
valid_dataset = ImageFolder(root='./splitted/valid', transform=transform_base)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

# 베이스라인 모델 설계

In [8]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(32,64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)
        self.fc1 = nn.Linear(4096, 512)
        self.fc2 = nn.Linear(512, 33)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = x.view(-1, 4096)
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)
    
model_base = Net().to(DEVICE)
optimizer = optim.Adam(model_base.parameters(), lr=0.001)


# 모델 학습/평가를 위한 함수

In [9]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (X, y) in enumerate(train_loader):
        X, y = X.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        y_hat = model(X)
        loss = F.cross_entropy(y_hat, y)
        loss.backward()
        optimizer.step()
        
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
        
            y_hat = model(X)
            test_loss = F.cross_entropy(y_hat, y, reduction='sum').item()
            
            pred = y_hat.max(1, keepdim=True)[1]
            
            correct += pred.eq(y.view_as(pred)).sum().item()
        
        test_loss /= len(test_loader.dataset)
        test_accuracy = 100. * correct / len(test_loader.dataset)
        return test_loss, test_accuracy            

# 모델 학습 실행하기

In [24]:
import time
import copy

def train_baseline(model, train_loader, val_loader, optimizer, num_epochs=30):
    best_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())
    
    for epoch in range(1, num_epochs +1):
        since = time.time()
        train(model, train_loader, optimizer)
        train_loss, train_acc = evaluate(model, train_loader)
        valid_loss, valid_acc = evaluate(model, valid_loader)
        
        if valid_acc > best_acc:
            best_acc = valid_acc
            best_model_wts = copy.deepcopy(model.state_dict())
        
        time_elapsed = time.time() - since
        print('----------------- epoch {} --------------'.format(epoch))
        print('train loss: {:.4f}, acc: {:.2f}%'.format(train_loss, train_acc))
        print('valid loss: {:.4f}, acc: {:.2f}%'.format(valid_loss, valid_acc))
        print('Completed in {:.0f}m, {:.0f}s'.format(time_elapsed //60, time_elapsed % 60))
    
    model.load_state_dict(best_model_wts)
    return model

base = train_baseline(model_base, train_loader, valid_loader, optimizer, EPOCH)
torch.save(base, 'baseline.pt')

----------------- epoch 1 --------------
train loss: 0.0034, acc: 82.12%
valid loss: 0.0051, acc: 79.63%
Completed in 0m, 15s
----------------- epoch 2 --------------
train loss: 0.0043, acc: 83.66%
valid loss: 0.0032, acc: 81.26%
Completed in 0m, 15s
----------------- epoch 3 --------------
train loss: 0.0037, acc: 86.62%
valid loss: 0.0031, acc: 83.40%
Completed in 0m, 15s
----------------- epoch 4 --------------
train loss: 0.0035, acc: 88.62%
valid loss: 0.0021, acc: 85.73%
Completed in 0m, 15s
----------------- epoch 5 --------------
train loss: 0.0021, acc: 89.75%
valid loss: 0.0032, acc: 86.48%
Completed in 0m, 15s
----------------- epoch 6 --------------
train loss: 0.0024, acc: 90.52%
valid loss: 0.0030, acc: 86.52%
Completed in 0m, 15s
----------------- epoch 7 --------------
train loss: 0.0022, acc: 91.86%
valid loss: 0.0019, acc: 88.05%
Completed in 0m, 15s
----------------- epoch 8 --------------
train loss: 0.0014, acc: 93.00%
valid loss: 0.0028, acc: 88.80%
Completed in 

# Transfer Learning

In [17]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((64,64)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
     'valid': transforms.Compose([
        transforms.Resize((64,64)),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
         
    ]),
}

data_dir = './splitted'

image_datasets = {x:ImageFolder(root=os.path.join(data_dir, x), transform=data_transforms[x]) for x in ['train', 'valid']}
dataloaders = {x:DataLoader(image_datasets[x], batch_size=BATCH_SIZE, shuffle=True, num_workers=4) for x in ['train', 'valid']}
dataset_sizes = {x:len(image_datasets[x])  for x in ['train', 'valid']}
class_names = image_datasets['train'].classes

## pre-trained 모델 불러오기

In [18]:
from torchvision import models
from torch.optim import lr_scheduler
# fc 바꿔치기
resnet = models.resnet50(pretrained=True)
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, len(class_names))   # to number of class
resnet.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, resnet.parameters()), lr=0.001)  # non-freeze만 학습 가능 
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1) # 7 epoch 마다 lr을 10% 씩 감소 

# 일부 layer freeze
ct = 0 
for child in resnet.children():   # resnet 10개층 가운데 초반 5개 freeze
    ct += 1
    if ct < 6:
        for param in child.parameters():
            param.requires_grad = False

In [19]:
import time
import copy


def train_resnet50(model, criterion, optimizer, scheduler, num_epochs=25):
    best_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())

    
    for epoch in range(1, num_epochs +1):
        print('----------------- epoch {} --------------'.format(epoch))
        since = time.time()
        
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0 # 모든 데이터의 lloss 총합
            running_corrects = 0
            
            for X, y in dataloaders[phase]:
                X, y = X.to(DEVICE), y.to(DEVICE)
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    y_hat = model(X)
                    _, preds = torch.max(y_hat, 1)
                    loss = criterion(y_hat, y)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                running_loss += loss.item() * X.size(0) # ???
                running_corrects += torch.sum(preds == y.data)
            
            if phase == 'train':
                scheduler.step()
                l_r = [x['lr'] for x in optimizer_ft.param_groups]
                print('learning rate: ', l_r)
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
        
            print('{} loss {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
        
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            
        time_elapsed = time.time() - since
        print('Completed in {:.0f}m, {:.0f}s'.format(time_elapsed //60, time_elapsed % 60))            
    
    print('Best val Acc: {:.4f}'.format(best_acc))
    
    model.load_state_dict(best_model_wts)
    
    return model    


In [20]:
base = train_resnet50(resnet, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=EPOCH)
torch.save(base, 'resnet50.pt')    

----------------- epoch 1 --------------
learning rate:  [0.001]
train loss 0.5962 Acc: 0.8199
valid loss 0.2984 Acc: 0.9036
Completed in 0m, 17s
----------------- epoch 2 --------------
learning rate:  [0.001]
train loss 0.2077 Acc: 0.9336
valid loss 0.1822 Acc: 0.9427
Completed in 0m, 16s
----------------- epoch 3 --------------
learning rate:  [0.001]
train loss 0.1710 Acc: 0.9441
valid loss 0.1847 Acc: 0.9412
Completed in 0m, 16s
----------------- epoch 4 --------------
learning rate:  [0.001]
train loss 0.1360 Acc: 0.9566
valid loss 0.1492 Acc: 0.9503
Completed in 0m, 16s
----------------- epoch 5 --------------
learning rate:  [0.001]
train loss 0.1135 Acc: 0.9637
valid loss 0.1588 Acc: 0.9476
Completed in 0m, 16s
----------------- epoch 6 --------------
learning rate:  [0.001]
train loss 0.0937 Acc: 0.9694
valid loss 0.1319 Acc: 0.9614
Completed in 0m, 16s
----------------- epoch 7 --------------
learning rate:  [0.0001]
train loss 0.0851 Acc: 0.9709
valid loss 0.1365 Acc: 0.958

# 모델 평가

In [12]:
# 베이스라인 모델용 전처리
transform_base = transforms.Compose([transforms.Resize(([64,64])), transforms.ToTensor()])
test_base = ImageFolder(root='./splitted/test', transform=transform_base)
test_loader_base = torch.utils.data.DataLoader(test_base, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

# transfer learning 모델용 전처리
transform_resnet = transforms.Compose([
        transforms.Resize((64,64)),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
         
    ])

test_resnet = ImageFolder(root='./splitted/test', transform=transform_resnet)
test_loader_resnet = torch.utils.data.DataLoader(test_resnet, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

## 베이스라인 모델 성능 평가

In [13]:
baseline = torch.load('baseline.pt')
baseline.eval()
test_loss, test_accuracy = evaluate(baseline, test_loader_base)
print('baseline test acc = {:.4f}'.format(test_accuracy))

baseline test acc = 93.8666


## Transfer learning 모델 성능 평가

In [14]:
resnet50 = torch.load('resnet50.pt')
resnet.eval()
test_loss, test_accuracy = evaluate(resnet50, test_loader_resnet)
print('Resnet test acc = {:.4f}'.format(test_accuracy))

FileNotFoundError: [Errno 2] No such file or directory: 'resnet50.pt'